# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [9]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [42]:
data = pd.read_csv('customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [11]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

In [12]:
customer_products.head(5)

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [13]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [14]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [15]:
prod_cust_pivot1 = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot1 = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [16]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [17]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
3,356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [18]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[264, 3535, 3317, 2503, 3305]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [19]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [20]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Butter - Unsalted,3,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0
3,Soup - Campbells Bean Medley,3,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0
6,Chicken - Soup Base,2,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [21]:
prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)

prod_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.003559,0.003927,0.004175,0.004102,0.003616,0.003767,0.003876,0.004024,0.004023,...,0.003794,0.003749,0.003846,0.004037,0.003839,0.004026,0.003908,0.003896,0.003860,0.003578
"Appetizer - Mini Egg Roll, Shrimp",0.003559,1.000000,0.003670,0.003697,0.003692,0.003413,0.003479,0.003900,0.003789,0.003486,...,0.003639,0.003481,0.003624,0.003751,0.003639,0.003508,0.003691,0.003749,0.003564,0.003475
Appetizer - Mushroom Tart,0.003927,0.003670,1.000000,0.004234,0.003849,0.003789,0.003819,0.004120,0.004110,0.003686,...,0.003920,0.003683,0.003812,0.004418,0.004155,0.004111,0.004095,0.004060,0.003937,0.003698
Appetizer - Sausage Rolls,0.004175,0.003697,0.004234,1.000000,0.004078,0.003974,0.003808,0.004169,0.003998,0.003837,...,0.003922,0.003853,0.003962,0.004551,0.004089,0.003872,0.004082,0.004020,0.003878,0.003718
Apricots - Dried,0.004102,0.003692,0.003849,0.004078,1.000000,0.003755,0.003722,0.004073,0.003895,0.003778,...,0.003760,0.003669,0.003869,0.004233,0.003836,0.003718,0.004018,0.003955,0.003811,0.003767


### Step 2: Get the products purchased for a specific customer of your choice.

In [54]:
data33=data[(data['CustomerID']==33) & (data["Quantity"]>0)]
data33.head(3)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
527,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1
1017,33,Lindsay,Santana,6509275,417,"Lamb - Pieces, Diced",1
3012,33,Lindsay,Santana,5638266,214,French Pastry - Mini Chocolate,1


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [23]:

nestedRecs = []

for product in data33['ProductName']:
    nestedRecs.append(prod_dist[product].sort_values(ascending=False)[1:6].index)

In [24]:
nestedRecs[0:3]

[Index(['Water - Spring Water 500ml', 'Raspberries - Fresh', 'Rabbit - Whole',
        'Cookies Cereal Nut', 'Sauce - Rosee'],
       dtype='object', name='ProductName'),
 Index(['Pork - Kidney', 'Beets - Mini Golden', 'Scallops 60/80 Iqf',
        'Wine - Vineland Estate Semi - Dry', 'Peas - Pigeon, Dry'],
       dtype='object', name='ProductName'),
 Index(['Scallops 60/80 Iqf', 'Sauce - Rosee', 'Water - Green Tea Refresher',
        'Bar - Granola Trail Mix Fruit Nut', 'Wine - Wyndham Estate Bin 777'],
       dtype='object', name='ProductName')]

In [25]:
lst = []
for item in nestedRecs:
    for i in item:
        if i not in data33['ProductName']:
            lst.append(i)
lst   


['Water - Spring Water 500ml',
 'Raspberries - Fresh',
 'Rabbit - Whole',
 'Cookies Cereal Nut',
 'Sauce - Rosee',
 'Pork - Kidney',
 'Beets - Mini Golden',
 'Scallops 60/80 Iqf',
 'Wine - Vineland Estate Semi - Dry',
 'Peas - Pigeon, Dry',
 'Scallops 60/80 Iqf',
 'Sauce - Rosee',
 'Water - Green Tea Refresher',
 'Bar - Granola Trail Mix Fruit Nut',
 'Wine - Wyndham Estate Bin 777',
 'Scallops 60/80 Iqf',
 'Wine - Magnotta - Belpaese',
 'Rabbit - Whole',
 'Pie Filling - Cherry',
 'Wine - Wyndham Estate Bin 777',
 'Rabbit - Whole',
 'Beef - Rib Eye Aaa',
 'Sauce - Rosee',
 'Water - Green Tea Refresher',
 'General Purpose Trigger',
 'Water - Green Tea Refresher',
 'Pie Filling - Cherry',
 'Scallops 60/80 Iqf',
 'Pickerel - Fillets',
 'Pork - Back, Short Cut, Boneless',
 'Water - Green Tea Refresher',
 'Sauce - Hollandaise',
 'Rabbit - Whole',
 'Pork - Loin, Center Cut',
 'Juice - Cranberry, 341 Ml',
 'Rabbit - Whole',
 'Water - Green Tea Refresher',
 'Pepper - Paprika, Hungarian',
 'Pant

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [26]:
import pandas as pd
pd.Series(lst).value_counts().head(5)

Water - Green Tea Refresher          33
Rabbit - Whole                       32
Scallops 60/80 Iqf                   32
Bar - Granola Trail Mix Fruit Nut    13
Wine - Wyndham Estate Bin 777        12
dtype: int64

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [27]:
allcust={}
customers = list(customer_products['CustomerID'].unique())

for customer in customers[0:3]:
    datax=data[(data['CustomerID']==customer) & (data["Quantity"]>0)]
    nestedRecs = []
    for product in datax['ProductName']:
        nestedRecs.append(prod_dist[product].sort_values(ascending=False)[1:6].index)
    lst = []
    for item in nestedRecs:
        for i in item:
            if i not in datax['ProductName']:
                lst.append(i)
    import pandas as pd
    top5=pd.Series(lst).value_counts().head(5).index
    allcust[customer]=top5
            
#.index to get rid of value count quantity


In [28]:
allcust

{33: Index(['Water - Green Tea Refresher', 'Rabbit - Whole', 'Scallops 60/80 Iqf',
        'Bar - Granola Trail Mix Fruit Nut', 'Wine - Wyndham Estate Bin 777'],
       dtype='object'),
 200: Index(['Water - Green Tea Refresher', 'Rabbit - Whole', 'Scallops 60/80 Iqf',
        'Bread - Hot Dog Buns', 'Bar - Granola Trail Mix Fruit Nut'],
       dtype='object'),
 264: Index(['Water - Green Tea Refresher', 'Rabbit - Whole', 'Scallops 60/80 Iqf',
        'Rice - Long Grain', 'Raspberries - Fresh'],
       dtype='object')}

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [29]:
item_recs = pd.DataFrame.from_dict(allcust, orient='index').reset_index()
item_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
item_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Water - Green Tea Refresher,Rabbit - Whole,Scallops 60/80 Iqf,Bar - Granola Trail Mix Fruit Nut,Wine - Wyndham Estate Bin 777
1,200,Water - Green Tea Refresher,Rabbit - Whole,Scallops 60/80 Iqf,Bread - Hot Dog Buns,Bar - Granola Trail Mix Fruit Nut
2,264,Water - Green Tea Refresher,Rabbit - Whole,Scallops 60/80 Iqf,Rice - Long Grain,Raspberries - Fresh


In [30]:
allcust

{33: Index(['Water - Green Tea Refresher', 'Rabbit - Whole', 'Scallops 60/80 Iqf',
        'Bar - Granola Trail Mix Fruit Nut', 'Wine - Wyndham Estate Bin 777'],
       dtype='object'),
 200: Index(['Water - Green Tea Refresher', 'Rabbit - Whole', 'Scallops 60/80 Iqf',
        'Bread - Hot Dog Buns', 'Bar - Granola Trail Mix Fruit Nut'],
       dtype='object'),
 264: Index(['Water - Green Tea Refresher', 'Rabbit - Whole', 'Scallops 60/80 Iqf',
        'Rice - Long Grain', 'Raspberries - Fresh'],
       dtype='object')}

## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [31]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [34]:
x = pd.DataFrame.from_dict(new_customer,orient='index').reset_index()

#refer to Tom example in recommender systems lab

In [53]:
prod_cust_pivot['x']=pd.Series(new_customer)
prod_cust_pivot['x'].fillna(0, inplace=True)
prod_cust_pivot.head(4)



CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,x
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0,0.0


In [48]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head(5)

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,x
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333,0.067975
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335,0.063707
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333,0.067640
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339,0.063985
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349,0.070847


In [52]:
similarities = cust_dist['x'].sort_values(ascending=False)[1:6]
similarities

CustomerID
1072    0.073478
412     0.070847
1534    0.070468
2503    0.069545
3885    0.069187
Name: x, dtype: float64

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [55]:
prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)

prod_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.003559,0.003927,0.004175,0.004102,0.003616,0.003767,0.003876,0.004024,0.004023,...,0.003794,0.003749,0.003846,0.004037,0.003839,0.004026,0.003908,0.003896,0.003860,0.003578
"Appetizer - Mini Egg Roll, Shrimp",0.003559,1.000000,0.003670,0.003697,0.003692,0.003413,0.003479,0.003900,0.003789,0.003486,...,0.003639,0.003481,0.003624,0.003751,0.003639,0.003508,0.003691,0.003749,0.003564,0.003475
Appetizer - Mushroom Tart,0.003927,0.003670,1.000000,0.004234,0.003849,0.003789,0.003819,0.004120,0.004110,0.003686,...,0.003920,0.003683,0.003812,0.004418,0.004155,0.004111,0.004095,0.004060,0.003937,0.003698
Appetizer - Sausage Rolls,0.004175,0.003697,0.004234,1.000000,0.004078,0.003974,0.003808,0.004169,0.003998,0.003837,...,0.003922,0.003853,0.003962,0.004551,0.004089,0.003872,0.004082,0.004020,0.003878,0.003718
Apricots - Dried,0.004102,0.003692,0.003849,0.004078,1.000000,0.003755,0.003722,0.004073,0.003895,0.003778,...,0.003760,0.003669,0.003869,0.004233,0.003836,0.003718,0.004018,0.003955,0.003811,0.003767


In [56]:
x

,index,0
0,Cookies - Assorted,3
1,Flavouring - Orange,3
2,Fenngreek Seed,1
3,Wine - White Cab Sauv.on,1
4,Bandage - Flexible Neon,3
5,Oil - Shortening - All - Purpose,2
6,Beef - Montreal Smoked Brisket,4
7,French Pastry - Mini Chocolate,4
8,Snapple Lemon Tea,5
9,"Pepper - White, Ground",2


In [57]:
nestedRecs = []

for product in x['index']:
    nestedRecs.append(prod_dist[product].sort_values(ascending=False)[1:6].index)

In [58]:
nestedRecs[0:3]

[Index(['Bread - Hot Dog Buns', 'Raspberries - Fresh', 'Cookies Cereal Nut',
        'Rabbit - Whole', 'Chocolate - Dark'],
       dtype='object', name='ProductName'),
 Index(['Pork - Loin, Center Cut', 'Water - Green Tea Refresher',
        'Water - Spring Water 500ml', 'Pasta - Orecchiette',
        'Scallops 60/80 Iqf'],
       dtype='object', name='ProductName'),
 Index(['Chinese Foods - Chicken', 'Cheese - Boursin, Garlic / Herbs',
        'Water - Green Tea Refresher', 'Vinegar - Sherry',
        'Scallops 60/80 Iqf'],
       dtype='object', name='ProductName')]

In [59]:
lst = []
for item in nestedRecs:
    for i in item:
        if i not in x['index']:
            lst.append(i)
lst   

['Bread - Hot Dog Buns',
 'Raspberries - Fresh',
 'Cookies Cereal Nut',
 'Rabbit - Whole',
 'Chocolate - Dark',
 'Pork - Loin, Center Cut',
 'Water - Green Tea Refresher',
 'Water - Spring Water 500ml',
 'Pasta - Orecchiette',
 'Scallops 60/80 Iqf',
 'Chinese Foods - Chicken',
 'Cheese - Boursin, Garlic / Herbs',
 'Water - Green Tea Refresher',
 'Vinegar - Sherry',
 'Scallops 60/80 Iqf',
 'Sausage - Breakfast',
 'Pork - Loin, Center Cut',
 'Scallops 60/80 Iqf',
 'Water - Green Tea Refresher',
 'Grouper - Fresh',
 'Barramundi',
 'Veal - Sweetbread',
 'Sherry - Dry',
 'Initation Crab Meat',
 'Lambcasing',
 'Appetizer - Sausage Rolls',
 'Pop Shoppe Cream Soda',
 'Beef - Prime Rib Aaa',
 'Scallops 60/80 Iqf',
 'Pie Filling - Cherry',
 'Sauce - Hollandaise',
 'Water - Green Tea Refresher',
 'Bar - Granola Trail Mix Fruit Nut',
 'Juice - Cranberry, 341 Ml',
 'Langers - Ruby Red Grapfruit',
 'Scallops 60/80 Iqf',
 'Sauce - Rosee',
 'Water - Green Tea Refresher',
 'Bar - Granola Trail Mix Frui

In [60]:
import pandas as pd
pd.Series(lst).value_counts().head(5)

Scallops 60/80 Iqf             6
Water - Green Tea Refresher    5
Raspberries - Fresh            3
Pork - Loin, Center Cut        3
Rabbit - Whole                 3
dtype: int64